# Data Preprocessing

This lab covers normalization, cross validation, regularization and dimension reduction.

In [ ]:
# Load Iris dataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
X,y=load_iris(return_X_y=True)
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)
X_train[:3]

### Normalization
Features are scaled to zero mean and unit variance:
$$z = ({x-\mu})/{\sigma}$$

### Missing Value Handling
In real datasets, missing entries are common.

In [ ]:
import numpy as np
X_missing = X_train.copy()
X_missing.ravel()[::40] = np.nan
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='mean')
X_imputed = imp.fit_transform(X_missing)


### Alternative Scaling
We can scale features to [0,1] using MinMaxScaler:

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
X_mm = mm.fit_transform(X_train)


In [ ]:
# Standardize features
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_scaled=sc.fit_transform(X_train)
X_scaled[:3]

### Cross Validation
We use cross-validation to estimate generalization performance.

In [ ]:
# Cross validation example
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
clf=LogisticRegression(max_iter=200)
cv_scores=cross_val_score(clf,X_scaled,y_train,cv=5)
cv_scores

### Regularization
Penalize large coefficients to reduce overfitting.

In [ ]:
# L2 regularization
from sklearn.linear_model import Ridge
ridge=Ridge(alpha=1.0)
ridge.fit(X_scaled,y_train)
ridge.score(sc.transform(X_test),y_test)

### Dimension Reduction
Principal Component Analysis (PCA) projects data to lower dimensions.

In [ ]:
# PCA example
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
X_pca=pca.fit_transform(X_scaled)
X_pca[:3]

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(X_pca[:,0], X_pca[:,1], c=y_train, cmap='viridis')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA Projection')
plt.show()

### Exercises
1. Evaluate a LogisticRegression with and without standardization.
2. Try `RobustScaler` and compare with `StandardScaler`.
3. Experiment with different values of `alpha` in Ridge regression.


### Hints
- Use `cross_val_score` for fair comparison.
- Visualization helps interpret PCA components.
